# MINN experiments

This notebook can be used to reproduced the results regarding the MINN presented in the paper.
Specifically, 
- the MINN-unbalanced
- MINN-c-balanced 
-  MINN-c-balanced (FBA reduced gem)
-  MINN-c-balance (FBA solution data) 
- MINN-divided loss
 
 You can select which experiment to reproduce by selecting the relative hydra configuration file in the second cell!

In [5]:
import numpy as np
from sklearn.model_selection import LeaveOneOut
import pandas as pd
from src.nn_model.amn_qp import *
from src.utils.hpo import hpo
from src.utils.import_data import *
from src.utils.import_GEM import *
from src.utils.training import *
from src.utils.plots import *
from src.utils.utils import *
from hydra import initialize, compose
from omegaconf import OmegaConf
import logging

# log hydra configuration file to choose which MINN configuration to run

In [ ]:
# hydra in jupyter notebook
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

config_dir = "conf" # Adjust path
initialize(config_path=config_dir, job_name="notebook_logging", version_base="1.1")

# HERE YOU CAN DECIDE WHICH CONFIGURATION FILE TO LOG
# MINN_c_balanced, MINN_unbalanced, MINN_c_balanced_FBA_fit_data, MINN_c_balanced_GEM_FBA_reduced, MINN_divided_loss

cfg = compose(config_name="MINN_c_balanced") 

# Step 3: Log the configuration
logger.info("Logging configuration:")
logger.info(OmegaConf.to_yaml(cfg))


# evaluation pipeline: loocv + kfold

In [ ]:
# No clearML when using jupyter
task= None
# reproduciblility
seed = cfg.seed
fix_random_seed(seed=seed)

# load gem and data
n_distribution= cfg.gem.n_total_reactions
X, y, Vin, fit_model, reference = load_ishii(seed=seed, 
                                                dataset=cfg.dataset.dataset_name,
                                                fluxes_removed_from_reference=cfg.dataset.fluxes_removed_from_reference,
                                                fluxes_to_add_input = cfg.dataset.fluxes_to_add_input,
                                                kos_genes= cfg.dataset.kos_genes,
                                                gem = cfg.gem.gem_name)

# Leave-One-Out Cross-Validation
loo = LeaveOneOut()

# Lists to store results for each LOO iteration
Q2_loo = []
MAE_loo = []
RMSE_loo = []
NE_loo = []

# List to store single left out predictions
Vref_true_loo = []
Vref_pred_loo = []
Vpred_final = []
list_SV_loss_per_sample = []

total_run= 29
count = 1
for train_idx, val_idx in loo.split(X):
    # Extract the training and validation data
    X_train, y_train, Vin_train = X[train_idx], y[train_idx], Vin[train_idx]
    X_val, y_val, Vin_val = X[val_idx], y[val_idx], Vin[val_idx]

    # kfolf cross validation 
    best_params = hpo(cfg, task=task, X_train=X_train, y_train=y_train, Vin_train=Vin_train, n_distribution=n_distribution, fit_model=fit_model)
    
    # train and test using the best hyperparameters found with the kfold
    results = train_test_evaluation(cfg, task, X_train, X_val, y_train, y_val, Vin_train, Vin_val, n_distribution, fit_model, best_params, loo_count=count)
    

    # Inverse transform targets to extract metrics
    Vref_pred_te = np.matmul(np.array(results["test"]['Vref_pred']), fit_model.Pref.T)
    Vref_true = results["test"]['Vref_true']
    
    
    # save metrics in the a list (one for each left out observation)
    Q2_value = r2_metric(np.array(Vref_true), Vref_pred_te)
    Q2_loo.append(Q2_value)

    MAE_value = np.mean(np.abs(Vref_true-Vref_pred_te))
    MAE_loo.append(MAE_value)

    RMSE_value = np.sqrt(np.mean(np.square(Vref_true-Vref_pred_te)))
    RMSE_loo.append(RMSE_value)

    NE_value = np.nan_to_num(np.linalg.norm(Vref_true - Vref_pred_te) / np.linalg.norm(Vref_true), posinf=0, neginf=0, nan=0)
    NE_loo.append(NE_value)

    Vref_true_loo.append(np.array(Vref_true).flatten())
    Vref_pred_loo.append(np.array(Vref_pred_te).flatten())
    Vpred_final.append(np.array(results["test"]['Vref_pred']).flatten())
    

    print(f'run {count}/{total_run}')
    print(f'Loss Train: {results["train"]["losses"]}')
    print(f'Loss Test: {results["test"]["losses"]}')
    print(f'Q²:{Q2_value}')
    print(f'MAE:{MAE_value}')
    print(f'RMSE {RMSE_value}')
    print(f'NE:{NE_value}')
    
    list_SV_loss_per_sample.append(results["test"]["losses"][1])
    count += 1
    



# Metrics calculation

In [ ]:
# Report table - DataFrame with index
if cfg.dataset.dataset_name=='ref_47_fluxes':
    df_true = pd.read_csv('data/ishii_data/fluxomics_iAF1260_reduced_split.csv')[cfg.dataset.metric_fluxes]
    
elif cfg.dataset.dataset_name=='ref_47_fluxes_fit':
    df_true = pd.read_csv('data/ishii_data/fluxomics_iAF1260_reduced_split_fit.csv')[cfg.dataset.metric_fluxes]
    
else:
    df_true = pd.read_csv('data/ishii_data/fluxomics_ecore_correct.csv')[cfg.dataset.metric_fluxes]


# Report table - DataFrame with index
df_V = pd.DataFrame(Vpred_final, columns=fit_model.reactions)[cfg.dataset.metric_fluxes]


metrics_df = metrics_table(df_true, df_V)
#add row with SV avg and std
avg_SV = np.array(list_SV_loss_per_sample).mean()
std_SV = np.array(list_SV_loss_per_sample).std()
metrics_df.loc["SV loss"] = [avg_SV, std_SV]

metrics_df